## 🐍 Python Descriptor

Um descritor em Python é um objeto que personaliza o acesso a atributos de outra classe, implementando um ou mais métodos especiais do Descriptor Protocol:

 - `__get__`(self, instance, owner)
Chamado ao ler instance.attr (ou via classe), controla como o valor é obtido.

 - `__set__`(self, instance, value)
Chamado ao atribuir instance.attr = value, permite validar, converter ou bloquear atribuições.

 - `__delete__`(self, instance)
Chamado ao executar del instance.attr, possibilitando interceptar ou impedir remoções.

 - `__set_name__`(self, owner, name) (desde o Python 3.6)
Executado automaticamente na criação da classe, recebe o nome do atributo a que o descritor foi atribuído, para uso interno (ex.: gerar self.private_name).

### Tipos de descritores

**Data descriptor**: implementa __set__ e/ou __delete__. Sempre tem precedência sobre valores no dicionário de instância.

**Non-data descriptor**: implementa apenas __get__. Só é chamado se não houver atributo com o mesmo nome em instance.__dict__.

#### 🧪 Exemplo 1 - Cache in Memory

In [ ]:
import pandas as pd

class LazyProperty:

    def __init__(self, func):
        self.func = func
        self.attr_name = func.__name__

    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        value = self.func(instance)
        setattr(instance, self.attr_name, value)
        print(f'instance: {instance}, self.attr_name: {self.attr_name})
        return value


class CSVDataPipeline:
    def __init__(self, csv_path: str):
        self.csv_path = csv_path

    @LazyProperty
    def raw_df(self) -> pd.DataFrame:
        print(f"🔄 Carregando dados de {self.csv_path}...")
        return pd.read_csv(self.csv_path)

    @LazyProperty
    def cleaned_df(self) -> pd.DataFrame:
        df = self.raw_df
        print("🧹 Limpando dados (drop de nulos)...")
        return df.dropna()

    def summary(self):
        df = self.cleaned_df
        return df.describe()

In [ ]:
%%timeit

pipeline = CSVDataPipeline("incoming/articles.csv")

df_raw = pipeline.raw_df
print(f"\nraw_df shape: {df_raw.shape}\n")

In [ ]:
%%time

df_raw_again = pipeline.raw_df
print(f"raw_df (cached) shape: {df_raw_again.shape}\n")

df_clean = pipeline.cleaned_df
print(f"\ncleaned_df shape: {df_clean.shape}\n")

print("Resumo estatístico do DataFrame limpo:")
pipeline.summary()

#### 🧪 Exemplo 2 - Cache in Disk and Memory

In [ ]:
from pathlib import Path
import pickle
import pandas as pd

class MemoizedDescriptor:

    def __init__(self, func):
        self.func = func
        self.name = func.__name__

    def __set_name__(self, owner, name):
        self.private_name = f"_{name}"

    def __get__(self, instance, owner):
        if instance is None:
            return self

        if hasattr(instance, self.private_name):
            return getattr(instance, self.private_name)

        cache_file = Path(instance.cache_dir) / f"{self.name}.pkl"

        if cache_file.exists():
            with open(cache_file, 'rb') as f:
                result = pickle.load(f)
            print(f" Carregando '{self.name}' de {cache_file}")
        else:
            print(f" Computando '{self.name}' e salvando em {cache_file}")
            result = self.func(instance)
            cache_file.parent.mkdir(parents=True, exist_ok=True)
            with open(cache_file, 'wb') as f:
                pickle.dump(result, f)

        setattr(instance, self.private_name, result)
        return result

class DataPipeline:
    """
    Exemplo de pipeline de dados que:
    1) Carrega um CSV
    2) Calcula estatísticas agrupadas
    Ambos os passos são memorizados e persistidos em disco.
    """
    def __init__(self, csv_path: str, cache_dir: str = "cache"):
        self.csv_path = csv_path
        self.cache_dir = cache_dir

    @MemoizedDescriptor
    def read_data(self) -> pd.DataFrame:

        print(f"Lendo CSV de {self.csv_path}...")
        return pd.read_csv(self.csv_path)

    @MemoizedDescriptor
    def stats(self) -> pd.DataFrame:
        df = self.read_data
        print(df.columns)
        return df.groupby('event_type')['value'].agg(['count', 'mean', 'std'])
    
    
    def unpersist(self, cache: str = 'both'):
        """
        Limpa o cache:
        - cache='memory': remove apenas o cache em memória
        - cache='disk': remove apenas os arquivos em disco
        - cache='both': remove ambos
        """
        valid = {'memory', 'disk', 'both'}
        if cache not in valid:
            raise ValueError(f"cache deve ser um de {valid}")

        # Limpeza de memória
        if cache in ('memory', 'both'):
            for attr in ['df', 'stats']:
                private = f"_{attr}"
                if hasattr(self, private):
                    delattr(self, private)
                    print(f"🗑️ Cache de memória removido: {attr}")

        # Limpeza de disco
        if cache in ('disk', 'both'):
            for name in ['df', 'stats']:
                cache_file = Path(self.cache_dir) / f"{name}.pkl"
                if cache_file.exists():
                    cache_file.unlink()
                    print(f"🗑️ Cache em disco removido: {cache_file}")    

#### 💻 Execução

In [ ]:
pipeline = DataPipeline("incoming/events.csv", cache_dir="cache")

In [ ]:
%%time    
# Primeira chamada: computa e persiste em cache/events.csv.stats.pkl
stats1 = pipeline.stats
print(stats1)

In [ ]:
%%time    
# Segunda chamada: carrega diretamente do cache sem recomputar
stats2 = pipeline.stats
print(stats2)

In [ ]:
# Limpa apenas memória
pipeline.unpersist(cache='memory')

In [ ]:
%%time    
# Terceira chamada: carrega diretamente do disco sem recomputar
stats3 = pipeline.stats
print(stats3)

In [ ]:
# Limpa apenas disco
pipeline.unpersist(cache='disk')

In [ ]:
%%time    

stats4 = pipeline.stats
print(stats4)

In [ ]:
# Limpa ambos
pipeline.unpersist(cache='both')

In [ ]:
%%time    

stats5 = pipeline.stats
print(stats5)

#### 🧪 Exemplo 3 - Controle de Acesso

In [3]:
import configparser
import pandas as pd

# Exemplo de arquivo permissions.ini:
#
# [permissions]
# data = admin,analyst
# metadata = admin

config = configparser.ConfigParser()
config.read('incoming/permissions.ini')


class AccessDescriptor:
    """
    Descriptor que controla acesso a um atributo com base no role do usuário.
    As permissões são definidas em permissions.ini, seção [permissions].
    """
    def __init__(self, attr_name: str):
        self.attr_name = attr_name
        perms = config['permissions'].get(attr_name, "")
        self.allowed_roles = [r.strip() for r in perms.split(',') if r.strip()]

    def __set_name__(self, owner, name):
        self.private_name = f"_{name}"

    def __get__(self, instance, owner):
        if instance is None:
            return self
        role = instance.user_role
        if role not in self.allowed_roles:
            raise PermissionError(
                f"Role '{role}' NÃO tem permissão para acessar '{self.attr_name}'"
            )
        return getattr(instance, self.private_name)

    def __set__(self, instance, value):
        role = instance.user_role
        if role not in self.allowed_roles:
            raise PermissionError(
                f"Role '{role}' NÃO tem permissão para modificar '{self.attr_name}'"
            )
        setattr(instance, self.private_name, value)


class Dataset:
    data = AccessDescriptor("data")
    metadata = AccessDescriptor("metadata")

    def __init__(self, csv_path: str, user_role: str):
        self.csv_path = csv_path
        self.user_role = user_role

        # Carrega imediatamente e armazena em atributos privados
        df = pd.read_csv(self.csv_path)
        object.__setattr__(self, "_data", df)
        object.__setattr__(self, "_metadata", {
            "columns": list(df.columns),
            "num_rows": len(df)
        })
        

In [7]:
from IPython.display import display

admin_ds = Dataset("incoming/events.csv", user_role="admin")
print("Admin pode ver dados:")
display(admin_ds.data.head())
print("Admin pode ver metadata:")
print(admin_ds.metadata, "\n")

Admin pode ver dados:


,timestamp,user_id,event_type,value
0,2025-06-01 00:00:00,1043,click,42.02
1,2025-06-01 01:00:00,1094,purchase,64.97
2,2025-06-01 02:00:00,1026,click,87.83
3,2025-06-01 03:00:00,1036,purchase,65.93
4,2025-06-01 04:00:00,1034,click,57.78


Admin pode ver metadata:
{'columns': ['timestamp', 'user_id', 'event_type', 'value'], 'num_rows': 100} 



In [8]:
# Usuário analyst: só tem acesso a data, não a metadata
analyst_ds = Dataset("incoming/events.csv", user_role="analyst")
print("Analyst pode ver dados:")
display(analyst_ds.data.head())
try:
    print("Analyst tentando ver metadata:")
    print(analyst_ds.metadata)
except PermissionError as e:
    print("Erro de permissão:", e)

Analyst pode ver dados:


,timestamp,user_id,event_type,value
0,2025-06-01 00:00:00,1043,click,42.02
1,2025-06-01 01:00:00,1094,purchase,64.97
2,2025-06-01 02:00:00,1026,click,87.83
3,2025-06-01 03:00:00,1036,purchase,65.93
4,2025-06-01 04:00:00,1034,click,57.78


Analyst tentando ver metadata:
Erro de permissão: Role 'analyst' NÃO tem permissão para acessar 'metadata'


#### 🧪Exemplo 4 - Logging Descriptor

In [ ]:
import logging
import pandas as pd

# Configuração básica do logger
logging.basicConfig(
    filename='outcoming/pipeline_access.log',
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

class LoggingDescriptor:

    def __set_name__(self, owner, name):
        self.name = name
        self.private_name = f"_{name}"

    def __get__(self, instance, owner):
        if instance is None:
            return self
        value = getattr(instance, self.private_name)
        logging.info(f"GET attribute '{self.name}' -> {repr(value)}")
        return value

    def __set__(self, instance, value):
        logging.info(f"SET attribute '{self.name}' <- {repr(value)}")
        setattr(instance, self.private_name, value)


class DataPipeline:
    """
    Pipeline de dados simples que loga cada vez que os DataFrames são
    lidos ou atualizados.
    """
    raw_data     = LoggingDescriptor()
    cleaned_data = LoggingDescriptor()

    def __init__(self, csv_path: str):
        # Carrega CSV e atribui a raw_data (gera log)
        df = pd.read_csv(csv_path)
        self.raw_data = df

    def clean(self):
        df = self.raw_data
        cleaned = df.dropna()
        self.cleaned_data = cleaned

    def summary(self):
        return self.cleaned_data.describe()


if __name__ == "__main__":
    pipeline = DataPipeline("incoming/events.csv")

    _ = pipeline.raw_data
    pipeline.clean()
    print(pipeline.summary())


#### 🧪 Exemplo - Construção de Constantes em Python

In [1]:
class Constant:
    # x
    def __set_name__(self, owner, name):
        self.private_name = f"_{name}"
        self.public_name  = name

    # print(x)
    def __get__(self, instance, owner):
        return getattr(instance, self.private_name)

    # X = 10
    def __set__(self, instance, value):
        if hasattr(instance, self.private_name):
            raise AttributeError(f"'{self.public_name}' é constante e não pode ser redefinida")
        setattr(instance, self.private_name, value)


class StaticValue:
    """Classe vazia; ganharemos constantes em tempo de execução."""
    pass


def register_constants(cls, *names: str):
    """
    Para cada nome fornecido, injeta um Constant() como descritor na classe.
    """
    for name in names:
        descriptor = Constant()
        # chama manualmente para inicializar private_name e public_name
        descriptor.__set_name__(cls, name)
        setattr(cls, name, descriptor)



# Exemplo de uso:
register_constants(StaticValue, "PI", "GRAVITY", "MY_FAVORITE")

const = StaticValue()
const.PI       = 3.14
const.GRAVITY  = 9.81
const.MY_FAVORITE = "Python"

print("PI =", const.PI)
print("GRAVITY =", const.GRAVITY)
print("MY_FAVORITE =", const.MY_FAVORITE)

# Tentativa de redefinir:
try:
    const.PI = 3.1415
except AttributeError as e:
    print("Erro:", e)


PI = 3.14
GRAVITY = 9.81
MY_FAVORITE = Python
Erro: 'PI' é constante e não pode ser redefinida
